In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # plotting library

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../data/raw/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

../data/raw/.gitkeep
../data/raw/train.csv
../data/raw/train.csv.zip


In [42]:
%%time
# Model parameters 
PATH =  '../data/raw/train.csv'
TARGET = 'Yards'
REMOVE = ['GameId','PlayId','Team','Yards','TimeHandoff','TimeSnap']

Wall time: 0 ns


In [10]:
%%time
# Define a seed for everything
def seed_everything(seed = 42):
    rn.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

Wall time: 0 ns


In [30]:
# Configure notebook display settings
pd.options.display.float_format = '{:,.1f}'.format
pd.set_option('display.max_columns', 20) 
pd.set_option('display.max_rows', 25) 

In [19]:
%%time
# Read the train dataset
train_df = pd.read_csv(PATH, low_memory = False)

Wall time: 3.93 s


In [20]:
%%time
# Read the train dataset
train_df.shape

Wall time: 0 ns


(509762, 49)

In [21]:
%%time
# Display the first few rows of the dataset
train_df.head()

Wall time: 999 µs


,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,...,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.9,34.8,1.7,1.1,0.4,82.0,177.2,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.7,32.6,0.4,1.4,0.0,27.6,198.7,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.0,33.2,1.2,0.6,0.3,3.0,202.7,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.5,27.7,0.4,0.5,0.0,359.8,105.6,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.3,35.4,1.8,2.4,0.2,12.6,164.3,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW


In [22]:
%%time
# Display the first few rows of the dataset
train_df.describe()

Wall time: 579 ms


,GameId,PlayId,X,Y,S,A,Dis,Orientation,Dir,NflId,...,Distance,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,DefendersInTheBox,Yards,PlayerWeight,Week,Temperature,Humidity
count,"509,762.0","509,762.0","509,762.0","509,762.0","509,762.0","509,762.0","509,762.0","509,744.0","509,748.0","509,762.0",...,"509,762.0","509,762.0","509,762.0","509,762.0","509,696.0","509,762.0","509,762.0","509,762.0","461,230.0","503,602.0"
mean,"2,017,594,430.7","20,175,944,309,347.4",60.0,26.8,2.6,1.7,0.3,180.2,179.9,"2,344,702.9",...,8.3,11.1,9.5,"2,419,203.8",6.9,4.2,253.6,9.1,60.4,55.6
std,"499,709.8","4,997,098,142.1",25.4,7.2,1.4,1.0,0.1,104.3,106.3,"641,887.1",...,3.8,10.3,9.4,"535,778.3",1.0,6.4,48.4,5.0,17.6,25.4
min,"2,017,090,700.0","20,170,907,000,118.0",0.7,1.9,0.0,0.0,0.0,0.0,0.0,222.0,...,1.0,0.0,0.0,234.0,1.0,-14.0,153.0,1.0,9.0,0.0
25%,"2,017,110,504.0","20,171,105,041,292.0",39.1,22.7,1.5,0.9,0.2,90.6,89.8,"2,532,865.0",...,6.0,3.0,0.0,"2,540,011.0",6.0,1.0,210.0,5.0,48.0,43.0
50%,"2,017,123,109.0","20,171,231,090,770.0",60.3,26.8,2.5,1.5,0.3,179.7,180.1,"2,543,765.0",...,10.0,9.0,7.0,"2,552,582.0",7.0,3.0,245.0,9.0,62.0,59.0
75%,"2,018,110,403.0","20,181,104,033,931.0",81.0,30.9,3.6,2.3,0.4,270.4,270.2,"2,555,343.0",...,10.0,17.0,14.0,"2,557,976.0",8.0,6.0,305.0,14.0,73.0,71.0
max,"2,018,123,015.0","20,181,230,154,157.0",116.8,56.5,9.4,14.7,1.4,360.0,360.0,"2,561,671.0",...,40.0,57.0,51.0,"2,561,468.0",11.0,99.0,380.0,17.0,97.0,100.0


In [23]:
%%time
# Display the first few rows of the dataset
train_df[TARGET].describe()

Wall time: 18 ms


count   509,762.0
mean          4.2
std           6.4
min         -14.0
25%           1.0
50%           3.0
75%           6.0
max          99.0
Name: Yards, dtype: float64

In [32]:
%%time
# Identify the nulls on datset
train_df.isnull().sum().sort_values(ascending = False)

Wall time: 426 ms


WindDirection             80234
WindSpeed                 67430
Temperature               48532
GameWeather               43648
StadiumType               32934
                          ...  
OffensePersonnel              0
Location                      0
NflIdRusher                   0
VisitorScoreBeforePlay        0
GameId                        0
Length: 49, dtype: int64

In [40]:
def describe_categorical(df, sample_fields = 5):
    ''' Describe categoricals datasets, in more details to have ageneral idea of the values on each column '''
    text_variables = []
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    df = df.select_dtypes(exclude = numerics)
    for col in df.columns:
        text_variables.append(col)
        pct_valid_data = 1 - (df[col].isnull().sum() / df.shape[0])
        unique_fields = list(set(df[col]))
        print('Variable Name: {:<18} % Data: {:0.2f} # Unique Fields:{:<6} Samples: {}'.format(col,pct_valid_data, len(unique_fields), unique_fields[:sample_fields]))
    return text_variables

In [41]:
train_df_categorical = describe_categorical(train_df, sample_fields = 6)

Variable Name: Team               % Data: 1.00 # Unique Fields:2      Samples: ['home', 'away']
Variable Name: DisplayName        % Data: 1.00 # Unique Fields:2230   Samples: ['Marqui Christian', 'Brandin Cooks', 'Sealver Siliga', 'David Morgan', 'Damien Wilson', 'Quinton Dunbar']
Variable Name: GameClock          % Data: 1.00 # Unique Fields:901    Samples: ['02:56:00', '10:56:00', '08:04:00', '06:31:00', '13:14:00', '07:51:00']
Variable Name: PossessionTeam     % Data: 1.00 # Unique Fields:32     Samples: ['NYJ', 'CLV', 'NE', 'CHI', 'SEA', 'PHI']
Variable Name: FieldPosition      % Data: 0.99 # Unique Fields:33     Samples: [nan, 'NYJ', 'CLV', 'NE', 'CHI', 'SEA']
Variable Name: OffenseFormation   % Data: 1.00 # Unique Fields:9      Samples: [nan, 'SINGLEBACK', 'WILDCAT', 'SHOTGUN', 'JUMBO', 'ACE']
Variable Name: OffensePersonnel   % Data: 1.00 # Unique Fields:56     Samples: ['1 RB, 4 TE, 0 WR', '0 RB, 2 TE, 3 WR', '6 OL, 1 RB, 2 TE, 1 WR', '2 QB, 2 RB, 1 TE, 1 WR', '6 OL, 2 RB, 1 TE